In [34]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, roc_auc_score, confusion_matrix
import sys
import os


# 1. LOAD PREDICTIONS
print(" Loading model predictions...")
file_path = '../data/processed/05_model_predictions.csv'

if not os.path.exists(file_path):
    print(f" Error: File not found at {file_path}")
    print("   Run '05_Model_Prediction.ipynb' first.")
    sys.exit(1)

df = pd.read_csv(file_path, index_col=0, parse_dates=True)
print(f"   Loaded {len(df)} rows.")
print(f"   Date Range: {df.index.min().date()} to {df.index.max().date()}")
print(f"   Pairs: {df['Pair_ID'].nunique()} unique pairs")


# 2. GLOBAL METRICS

print("GLOBAL MODEL PERFORMANCE")


y_true = df['Target_Direction'].values
# Ensure binary targets
y_true_binary = (y_true > 0.5).astype(int)

models = ['Ridge', 'LSTM']
metrics_list = []

for model in models:
    # Get columns dynamically
    pred_col = f"{model}_Pred"
    prob_col = f"{model}_Prob" if f"{model}_Prob" in df.columns else pred_col
    
    y_pred = df[pred_col].values
    y_prob = df[prob_col].values
    
    # Binary conversion if needed
    y_pred_bin = (y_pred > 0.5).astype(int)
    
    # Calculate Core Metrics
    acc = accuracy_score(y_true_binary, y_pred_bin)
    prec = precision_score(y_true_binary, y_pred_bin, zero_division=0)
    rec = recall_score(y_true_binary, y_pred_bin, zero_division=0)
    f1 = f1_score(y_true_binary, y_pred_bin, zero_division=0)
    
    try:
        auc = roc_auc_score(y_true_binary, y_prob)
    except:
        auc = 0.5
        
    metrics_list.append({
        'Model': model,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1_Score': f1,
        'ROC_AUC': auc
    })

metrics_df = pd.DataFrame(metrics_list).set_index('Model')
print(metrics_df.round(4).to_string())


# Identify Winner
winner = metrics_df['Precision'].idxmax() # Precision is king in trading
print(f"PREFERRED MODEL: {winner} (Highest Precision)")


# 3. PER-PAIR BREAKDOWN (Critical for later Trading backtest)


print("PERFORMANCE BY PAIR")
print("Which pairs the models are actually good at predicting?\n")

pair_stats = []

for pair in df['Pair_ID'].unique():
    pair_data = df[df['Pair_ID'] == pair]
    y_p_true = pair_data['Target_Direction'].values
    
    stats = {'Pair_ID': pair, 'Samples': len(pair_data)}
    
    for model in models:
        pred_col = f"{model}_Pred"
        y_p_pred = (pair_data[pred_col].values > 0.5).astype(int)
        
        # We focus on Accuracy and Precision for per-pair selection
        acc = accuracy_score(y_p_true, y_p_pred)
        prec = precision_score(y_p_true, y_p_pred, zero_division=0)
        
        stats[f'{model}_Acc'] = acc
        stats[f'{model}_Prec'] = prec
        
    pair_stats.append(stats)

pair_df = pd.DataFrame(pair_stats).set_index('Pair_ID')
pair_df = pair_df.sort_values(by='LSTM_Prec', ascending=False)

# Display Top 5 Pairs
print("TOP 5 PAIRS (by LSTM Precision):")
print(pair_df[['Samples', 'Ridge_Prec', 'LSTM_Prec']].head(5).round(4).to_string())

print("\nBOTTOM 5 PAIRS (Avoid these):")
print(pair_df[['Samples', 'Ridge_Prec', 'LSTM_Prec']].tail(5).round(4).to_string())


# 4. MODEL AGREEMENT & HYBRID SIGNAL
print("MODEL CONSENSUS ANALYSIS")


# Create a "Hybrid" signal: Trade ONLY if BOTH models say YES (1)
df['Hybrid_Pred'] = ((df['Ridge_Pred'] == 1) & (df['LSTM_Pred'] == 1)).astype(int)

hybrid_acc = accuracy_score(y_true_binary, df['Hybrid_Pred'])
hybrid_prec = precision_score(y_true_binary, df['Hybrid_Pred'], zero_division=0)
trade_count = df['Hybrid_Pred'].sum()

print(f"Strategy: Trade only when BOTH models agree (Converge)")
print(f"   - Trades Triggered: {trade_count} / {len(df)}")
print(f"   - Hybrid Accuracy:  {hybrid_acc:.4f}")
print(f"   - Hybrid Precision: {hybrid_prec:.4f}")

if hybrid_prec > metrics_df.loc['LSTM', 'Precision']:
    print("\n INSIGHT: The Hybrid consensus improves Precision! Use this for safer trades.")
else:
    print("\n INSIGHT: Hybrid does not improve Precision. Stick to the single best model.")



# 5. SAVE REPORTS

output_metrics = '../data/processed/06_global_metrics.csv'
output_pairs = '../data/processed/06_pair_performance.csv'

metrics_df.to_csv(output_metrics)
pair_df.to_csv(output_pairs)

print(f"Saved global metrics to: {output_metrics}")
print(f"Saved pair performance to: {output_pairs}")
print("\n Evaluation Complete.")

 Loading model predictions...
   Loaded 2405 rows.
   Date Range: 2024-01-17 to 2025-12-15
   Pairs: 5 unique pairs
GLOBAL MODEL PERFORMANCE
       Accuracy  Precision  Recall  F1_Score  ROC_AUC
Model                                                
Ridge    0.5543     0.5706  0.5670    0.5688   0.5538
LSTM     0.5638     0.5650  0.6905    0.6214   0.5803
PREFERRED MODEL: Ridge (Highest Precision)
PERFORMANCE BY PAIR
Which pairs the models are actually good at predicting?

TOP 5 PAIRS (by LSTM Precision):
          Samples  Ridge_Prec  LSTM_Prec
Pair_ID                                 
AME-ITW       481      0.5869     0.6139
CMS-DUK       481      0.5894     0.5922
MS-STT        481      0.5801     0.5647
AIG-CB        481      0.5681     0.5426
FITB-PNC      481      0.5345     0.5263

BOTTOM 5 PAIRS (Avoid these):
          Samples  Ridge_Prec  LSTM_Prec
Pair_ID                                 
AME-ITW       481      0.5869     0.6139
CMS-DUK       481      0.5894     0.5922
MS-STT  

In [38]:
#save the models performance metrics and pair perfomances to outputs folder
import os

# 1. Define the paths
source_path = '../data/processed/06_model_metrics.csv'
output_dir = '../results'
output_path = os.path.join(output_dir, 'models_metrics.csv')

# 2. Load the CSV 
df_final = pd.read_csv(source_path)

# 4. Save the table to the output folder
df_final.to_csv(output_path, index=False)

print(f"Table successfully saved to: {output_path}")

source_path = '../data/processed/06_pair_performance.csv'
output_dir = '../results'
output_path = os.path.join(output_dir, 'pair_performance.csv')
df_final = pd.read_csv(source_path)
# 4. Save the table to the output folder
df_final.to_csv(output_path, index=False)
print(f"Table successfully saved to: {output_path}")



Table successfully saved to: ../results\models_metrics.csv
Table successfully saved to: ../results\pair_performance.csv
